In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Third Party
from datasets import load_dataset
from openai import OpenAI
from rich import print
from rich.panel import Panel
from sklearn.metrics import classification_report

# First Party
from sdg_hub import Flow, FlowMetadata, BlockRegistry

import nest_asyncio
nest_asyncio.apply()

# Classifying news articles


In this tutorial, you’ll learn how to create your own custom data generation flow using SDG Hub. This notebook will walk you through all the essential pieces to make your own flow using `sdg_hub` for any use-case using the fundamental components of sdg_hub: `Blocks` and `Flows`

As an example use-case, we will pick news classification. Classification is a fundamental task in machine learning, where the goal is to assign predefined categories to input data. To address the classic machine learning use-case of news or text classification, we will use sdg_hub and leverage a language model to **classify news articles** with topic labels — specifically using the [AG News dataset](https://huggingface.co/datasets/fancyzhx/ag_news) from Hugging Face.

We’ll go step by step through a progressively improving flow. Each stage builds on the previous one, giving you a practical sense of how you can evolve your flow from using simple heuristics to highly customized and reliable data generation, using different inference paradigms such as self assessment.

### 🔍 Understand the Task
Before we write any prompts or code, we’ll take time to understand what we want the model to do. For this exercise, the task is **text classification** — assigning one of 4 possible categories (e.g., "World", "Sports", "Sci/Tech", "Business") to a given news article

### 🛠️ Build a Basic Annotation Flow and learn the `sdg_hub` way
We’ll start by creating a minimal flow that simply prompts the model to generate topic labels on the unlabeled data. This will use default prompts, simply populating the prompt with the text and asking the model to generate one of the 4 possible labels, with no examples.

### 🎯 Improve with Assessment and Iteration
Next, we’ll refine the flow by adding an assessment step. Iterations and self verification on a task often lead to better performance

Let’s get started by loading a sample of the dataset

In [ ]:
dataset = load_dataset("fancyzhx/ag_news")

train_data = dataset["train"].shuffle(seed=42).select(range(500))
test_data = dataset["test"].shuffle(seed=42).select(range(100))

# map the labels to the category names
label_map = train_data.features['label'].names

train_data = train_data.map(lambda x: {"category": label_map[x["label"]]})
test_data = test_data.map(lambda x: {"category": label_map[x["label"]]})

In [ ]:
# Group examples by category
examples_by_category = {}
for item in train_data:
    category = item['category']
    if category not in examples_by_category:
        examples_by_category[category] = []
    examples_by_category[category].append(item['text'])

# Print one example from each category in a panel
for category, examples in examples_by_category.items():
    print(Panel(examples[0], title=f"Category: {category}", expand=False))


## Simple Data Annotation Pipeline

In this section, we’ll create our **first working flow** to perform classification using a language model. The goal is to understand the building blocks of `sdg_hub` and how we can employ them to get a language model to classify a given text.

### Recap: How `sdg_hub` Works

```mermaid
flowchart LR
    A[Flow] --> B[Blocks] --> C[Prompts]
    C --> D[Generated Data]
```

# Building a Simple Classification Flow

### Discover Blocks for us to use



In [ ]:
BlockRegistry.discover_blocks()

It seems all the functionality we are interested in, such as building a prompt, chatting with an llm and parsing its output are under the `llm` category in sdg_hub. Lets start there.

In [ ]:
from sdg_hub.core.blocks.llm import PromptBuilderBlock, LLMChatBlock, TextParserBlock, LLMParserBlock

### Creating the required blocks

To get started, we'll construct the simplest possible flow for text classification using SDG Hub. We will focus on 3 main blocks that will often appear as a triplet while using `sdg_hub`

1. **Prompt Builder Block**: Converts each input text into a prompt formatted for the LLM. The important input argument to keep in mind for  `PromptBuilderblock` is the `prompt_config_path` which is where the prompt template is saved. Any prompt engineering we would want to do would be done in such a prompt template.
2. **LLM Chat Block**: Sends the prompt to the language model and receives its response (the predicted label).
3. **Text Parser Block**: Extracts the final label from the LLM's output.

This setup results in a single LLM interaction per sample, forming a minimal classification pipeline.

We are going to be using the simple prompt that can be found in `news_articles_classification_prompt.yaml`

In [ ]:
promptbuilderblock_1 = PromptBuilderBlock(block_name='annotation_prompt_builder', input_cols=['text'], output_cols=['annotation_prompt'], prompt_config_path="news_classification_prompt.yaml", format_as_messages=True)
llmchatblock_1 = LLMChatBlock(block_name='annotation_llm_chat_block', input_cols=['annotation_prompt'], output_cols=['raw_output'], temperature=0.0, max_tokens=5, extra_body={'guided_choice': ['World', 'Sports', 'Business', 'Sci/Tech']}, async_mode=True)
llmparserblock_1 = LLMParserBlock(block_name='annotation_llm_parser_block', input_cols=['raw_output'], extract_content=True, expand_lists=True)
textparserblock_1 = TextParserBlock(block_name='annotation_text_parser_block', input_cols=['annotation_llm_parser_block_content'], output_cols=['output'], start_tags=[''], end_tags=[''])

### Designing the `Flow`

The `Flow` class is at the heart of SDG Hub. Simply put, a `Flow` is a chain of `Blocks` that get executed sequentially. Here, we will simply chain our PromptBuilder -> LLMChatBlock -> TextParser, in that order:

```mermaid
flowchart LR
    subgraph Flow
        direction LR
        A[PromptBuilderBlock] --> B[LLMChatBlock] --> C[TextParserBlock]
    end
```



In [ ]:
flow = Flow(blocks=[promptbuilderblock_1, llmchatblock_1, llmparserblock_1, textparserblock_1], metadata=FlowMetadata(name="annotation_flow", description="A flow for news article classification", author="sdg_hub"))

### Set the model configs for the `Flow`

In SDG Hub, model details such as the API base URL, the API Key (if any) and the model name are set at a Flow level using the `set_model_config` method as shown. The `model` parameter accepts a string in the format of "`provider`/`model_name`". Here our `provider` is 'hosted_vllm' as we are using a locally hosted model through vllm, and the model name is "meta-llama/Llama-3.3-70B-Instruct"

We must set the `api_base` parameter and point it to where the model endpoint can be found, in this case, `http://localhost:8000/v1`

In [ ]:
# flow.set_model_config(model="hosted_vllm/meta-llama/Llama-3.3-70B-Instruct", api_base="http://localhost:8000/v1", api_key="")

flow.set_model_config(model="hosted_vllm/qwen3-8b", api_base="http://localhost:8101/v1", api_key="empty")


### Time to generate!

In sdg_hub, the way to generate data is very simple. we simply use the `generate` method from `Flow`. At its simplest form, all the `generate` method needs is the input dataset to operate on. Additionally, we can pass runtime parameters for each block as well, if we wish to override any of the block specific model configs.

In [ ]:
generated_data = flow.generate(test_data)

### Evaluation

Now that we’ve generated synthetic labels using our simple classification flow, it’s time to evaluate how well the model performed. The goal of this section is to compare the predicted labels against the **true labels** from the dataset using standard classification metrics (precision, recall, f-1 score and classification accuracy)

We’ll use `sklearn.metrics.classification_report`, which provides precision, recall, F1-score, and support for each class.


In [ ]:
print(classification_report(generated_data["category"], generated_data["output"]))

## Introducing an Assessment step

Our initial flow used a one step approach — the model was given the task, a fixed label set, and some input text. While this baseline gives us a useful starting point, it has clear limitations:

- The model may rely on generic heuristics or surface patterns that don’t generalize well.
- It can confuse similar categories (e.g., "World" vs. "Business") without knowing how they're typically used.
- Without guidance, the model may underperform on edge cases or ambiguous queries.


### What is Assessment

With an assessment step, we will call to the same LLM, but this time, we provide the LLM with its own previous categorization label, and the original text. We will prompt the LLM to think about the original prediction, and give it context about challening cases
In this manner, we can elicit critical judgement from the model about its own prior classification decision. This type of additional context can be useful in the next iteration.


### What We’ll Do Next

We’ll now enhance our flow by introducing another chain of `PromptBuilder` -> `LLMChatBlock` -> `TextParserBlock` whose purpose is to pass the (original text +  prediction) to the LLM and obtain a verification or assessment of the prediction.


```mermaid
flowchart LR
 subgraph Flow1[Initial Classification]
 direction LR
 A[PromptBuilderBlock] --> B[LLMChatBlock] --> C[TextParserBlock]
 end
 subgraph Flow2[Assessment]
 direction LR
 D[PromptBuilderBlock_Assessment] --> E[LLMChatBlock_Assessment] --> F[TextParserBlock_Assessment]
 end
 
 C --> D
```


We will investigate if this catches any of the mis-classifications, and get an idea of how well our verification prompting works!

In [ ]:
promptbuilderblock_assessment = PromptBuilderBlock(block_name='verifier_prompt_builder', input_cols=['text', 'output'], output_cols=['assessment_prompt'], prompt_config_path="news_classification_assessment_prompt.yaml", format_as_messages=True)
llmchatblock_assessment = LLMChatBlock(block_name='verifier_llm_chat_block', input_cols=['assessment_prompt'], output_cols=['raw_assessment_output'], async_mode=True)
llmparserblock_assessment = LLMParserBlock(block_name='verifier_llm_parser_block', input_cols=['raw_assessment_output'], extract_content=True, expand_lists=True)
textparserblock_assessment = TextParserBlock(block_name='verifier_text_parser_block', input_cols=['verifier_llm_parser_block_content'], output_cols=['assessment_output'], start_tags=[''], end_tags=[''])

flow = Flow(blocks=[promptbuilderblock_1, llmchatblock_1, llmparserblock_1, textparserblock_1, promptbuilderblock_assessment, llmchatblock_assessment, llmparserblock_assessment, textparserblock_assessment], metadata=FlowMetadata(name="annotation_flow", description="A flow for news article classification", author="sdg_hub"))
# flow.set_model_config(model="hosted_vllm/meta-llama/Llama-3.3-70B-Instruct", api_base="http://localhost:8000/v1", api_key="")
flow.set_model_config(model="hosted_vllm/qwen3-8b", api_base="http://localhost:8101/v1", api_key="empty")



generated_data = flow.generate(test_data)

In [ ]:
generated_data_pd = generated_data.to_pandas()
mislabeled_samples = generated_data_pd[generated_data_pd["category"] != generated_data_pd["output"]]

print(Panel(mislabeled_samples.iloc[0]['assessment_output'], title="Assessment"))
print(Panel(str(mislabeled_samples.iloc[0]['category']), title="Ground truth label"))

Great! Now we can see that the assessment step is working good, especially on the misclassified samples as shown above. The above is a hard example which has slipped past our original classification flow, but was caught by our assessment step's critical judgement.

### Revising the Classifications

We will now create our final revision step, which will take the results of the initial prediction and the assessment steps and pass it onto the LLM once again for a revised attempt at classifying the same input text. The flow can be imagined like so:

```mermaid
flowchart LR
 subgraph Flow1[Initial Classification]
 direction LR
 A[PromptBuilderBlock] --> B[LLMChatBlock] --> C[TextParserBlock]
 end
 subgraph Flow2[Assessment]
 direction LR
 D[PromptBuilderBlock_Assessment] --> E[LLMChatBlock_Assessment] --> F[TextParserBlock_Assessment]
 end
 subgraph Flow3[Revised Classification]
 direction LR
 G[PromptBuilderBlock_Revision] --> H[LLMChatBlock_Revision] --> I[TextParserBlock_Revision]
 end
 
 C --> D
 F --> G
```

In [ ]:
promptbuilderblock_revision = PromptBuilderBlock(block_name='revised_prompt_builder', input_cols=['text', 'output', 'assessment_output'], output_cols=['revised_prompt'], prompt_config_path="revise_news_classification_prompt.yaml", format_as_messages=True)
llmchatblock_revision = LLMChatBlock(block_name='revised_llm_chat_block', input_cols=['revised_prompt'], output_cols=['raw_revised_output'], temperature=0.0, max_tokens=5, extra_body={'guided_choice': ['World', 'Sports', 'Business', 'Sci/Tech']}, async_mode=True)
llmparserblock_revision = LLMParserBlock(block_name='revised_llm_parser_block', input_cols=['raw_revised_output'], extract_content=True, expand_lists=True)
textparserblock_revision = TextParserBlock(block_name='revised_text_parser_block', input_cols=['revised_llm_parser_block_content'], output_cols=['revised_output'], start_tags=[''], end_tags=[''])

flow = Flow(blocks=[promptbuilderblock_1, llmchatblock_1, llmparserblock_1, textparserblock_1, promptbuilderblock_assessment, llmchatblock_assessment, llmparserblock_assessment, textparserblock_assessment, promptbuilderblock_revision, llmchatblock_revision, llmparserblock_revision, textparserblock_revision], metadata=FlowMetadata(name="news_classification_flow", description="A flow for news article classification with assessment and revision", author="sdg_hub"))
# flow.set_model_config(model="hosted_vllm/meta-llama/Llama-3.3-70B-Instruct", api_base="http://localhost:8000/v1", api_key="")
flow.set_model_config(model="hosted_vllm/qwen3-8b", api_base="http://localhost:8101/v1", api_key="empty")

In [ ]:
generated_data = flow.generate(test_data)

In [ ]:
print(classification_report(generated_data["category"], generated_data["revised_output"]))

🔥 We improved the results drastically! Let us take a look at the number of mislabeled samples before and after the assessment + revision steps


In [ ]:
generated_data_pd = generated_data.to_pandas()
num_mislabeled_output = (generated_data_pd["category"] != generated_data_pd["output"]).sum()
num_mislabeled_revised = (generated_data_pd["category"] != generated_data_pd["revised_output"]).sum()
print(f"Number of mislabeled samples (original output): {num_mislabeled_output}")
print(f"Number of mislabeled samples (revised output): {num_mislabeled_revised}")


Great, we whave now improved the classification accuracy of our system by augmenting our naive classification flow by adding an assessment followed by a revision step


### Export the flow to yaml form


In [ ]:
flow.to_yaml("news_classification_flow.yaml")

## ✅ Summary: What You’ve Learned

In this tutorial, you learned how to create your own flow for a custom use-case using `sdg_hub`, using the fundamental components: `Flow` and `Block`. You also learned how to create and structure the prompts. You learned how to design an assessment or a judgement step in order to improve the performance of the overall system. You started from scratch and evolved it into a robust, high-accuracy system.

## 🚀 What’s Next?

* Prompt Engineer! -  You can add examples for classifications directly in the classification steps and see how this improves the performance. In-context examples are extremely effective at aligning the model's outputs to the task at hand
* Try it out on your own data!